In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt


In [ ]:
dir_SLAND = '/Trendy/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_ctrs  = '/Trendy/Data/data_ancillary/info_countries/'
dir_out   = '/Trendy/Data/SLAND_Trendy-v10_S2_countries/'


## Calculation

In [ ]:
models = ['CABLE-POP', 'CLASSIC', 'CLASSIC-N', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']

gridsize = '720x1440'

#Read ISO codes for countries, IPCC countries, and conversions between ISO alpha-3 codes from IPCC and ISO numeric
fname_ctrs_ISO   = dir_ctrs + 'wrld_cntrs_BLUE_TN_upd.nc'
fname_IPCC_codes = dir_ctrs + 'IPCC_regions.xlsx'
fname_ISO_num    = dir_ctrs + 'iso_codes_alpha_numeric.xlsx'
data_ctrs_ISO   = xr.open_dataset(fname_ctrs_ISO)
data_IPCC_codes = pd.read_excel(fname_IPCC_codes, sheet_name='region_classification', header=0, usecols=[0, 1, 3])
data_alph_num   = pd.read_excel(fname_ISO_num, header=0)

#Create dicts for storing data
SLAND_ctrs = dict()

#Define output file name
fname_out = dir_out + 'SLAND_S2_DGVMS_countries_ISOcode_vRemapSLAND.xlsx'
if os.path.exists(fname_out): os.remove(fname_out)

#Create xlsx-file (it will be filled at end of loop with country data from every model)
with pd.ExcelWriter(fname_out) as writer:
    
    #Loop over models
    for model in models:
        
        print(model)
        
        #Get file name for SLAND
        fnames = [file for file in os.listdir(dir_SLAND) if (model + '_' in file) and (gridsize in file) and ('SLAND_NBP' in file) and ('NBPPFT' not in file)]
        if len(fnames)!=1:  sys.exit('Filename not unique')
            
        #Read SLAND data
        fname = dir_SLAND + fnames[0]
        data_SLAND = xr.open_dataset(fname)

        #Get lat and lon names
        if 'latitude' in data_SLAND.dims:  lat_name, lon_name = 'latitude', 'longitude'
        else:                              lat_name, lon_name = 'lat', 'lon'
        
        #Check that model grid and country grid agree
        check_lat = np.max(np.abs(data_SLAND[lat_name].values - data_ctrs_ISO.lat.values))
        check_lon = np.max(np.abs(data_SLAND[lon_name].values - data_ctrs_ISO.lon.values))
        if check_lat>0.01:  sys.exit('Latitudes do not agree')
        if check_lon>0.01:  sys.exit('Longitudes do not agree')
            
        #Re-index if there are small deviations in lat and lon
        if (check_lat!=0) or (check_lon!=0):
            data_SLAND = data_SLAND.reindex({lat_name: data_ctrs_ISO['lat'], lon_name: data_ctrs_ISO['lon']}, method='nearest')       
            
        #Loop over all country codes
        for i, iso_alpha3 in enumerate(data_IPCC_codes['ISO']):
            
            if np.mod(i, 50)==0:
                print('  -run ' + str(i+1) + ' of ' + str(len(data_IPCC_codes['ISO'])))
            
            #Get numbeic ISO code of country
            iso_numeric = data_alph_num['Numeric'][data_alph_num['Alpha-3 code']==iso_alpha3].values[0]

            #Select country in country mask
            mask_ISO = data_ctrs_ISO.ISOcode==iso_numeric
            
            #Get SLAND sum in selected country
            data_sel = data_SLAND.where(mask_ISO).sum(('lat', 'lon'))
            
            #Convert to Tg C/year
            data_sel = 1000 * data_sel
            
            #Save values in dict
            SLAND_ctrs[iso_alpha3] = data_sel.SLAND.values

        #Special cases for certain IPCC countries
        SLAND_ctrs['SXM'] = SLAND_ctrs['MAF']                                          # Saint Martin is French part of island with Sint Maarten (Dutch part) -> same values are counted for both
        SLAND_ctrs['ANT'] = SLAND_ctrs['BES'] + SLAND_ctrs['CUW'] + SLAND_ctrs['SXM']  # Netherlands Antilles (Bonaire, Saint Eustatius & Saba + Curacao + Sint Maarten)
  
        #Convert data to data frame (and sort by country name)
        SLAND_ctrs_df = pd.DataFrame(SLAND_ctrs, index=data_SLAND.time.dt.year)
        SLAND_ctrs_df = SLAND_ctrs_df.reindex(sorted(SLAND_ctrs_df.columns), axis=1)
        
        #Adde units in first cell
        SLAND_ctrs_df = SLAND_ctrs_df.rename_axis('unit: TG C/year')
        
        #Create sheet in xlsx for every model and store country data
        SLAND_ctrs_df.to_excel(writer, sheet_name=model + '_SLAND_IPCC_ctrs', index=True, header=True, float_format='%.6f')
